In [1]:
library(clusterProfiler)
library(org.Hs.eg.db)
library(stringr)
library(edgeR)
library(Seurat)



clusterProfiler v3.18.1  For help: https://guangchuangyu.github.io/software/clusterProfiler

If you use clusterProfiler in published research, please cite:
Guangchuang Yu, Li-Gen Wang, Yanyan Han, Qing-Yu He. clusterProfiler: an R package for comparing biological themes among gene clusters. OMICS: A Journal of Integrative Biology. 2012, 16(5):284-287.


Attaching package: ‘clusterProfiler’


The following object is masked from ‘package:stats’:

    filter


Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are

In [2]:
as_matrix <- function(mat){
  
  tmp <- matrix(data=0L, nrow = mat@Dim[1], ncol = mat@Dim[2])
  
  row_pos <- mat@i+1
  col_pos <- findInterval(seq(mat@x)-1,mat@p[-1])+1
  val <- mat@x
  
  for (i in seq_along(val)){
    tmp[row_pos[i],col_pos[i]] <- val[i]
  }
  
  row.names(tmp) <- mat@Dimnames[[1]]
  colnames(tmp) <- mat@Dimnames[[2]]
  return(tmp)
}

# load data

In [4]:
obj_Adipocyte <- readRDS("/stor/public/chenyx/HHCAd/cell_type_organized_fetal/Adipocyte.rds")

In [5]:
obj_Cardiomyocyte_cell <- readRDS("/stor/public/chenyx/HHCAd/cell_type_organized_fetal/Cardiomyocyte cell.rds")
obj_Cardiomyocyte_cell <- subset(obj_Cardiomyocyte_cell, subset = cell_id %in% obj_Cardiomyocyte_cell$cell_id[sample(1:dim(obj_Cardiomyocyte_cell)[2],2000)])

In [6]:
obj_merge <- merge(obj_Adipocyte,obj_Cardiomyocyte_cell)

In [7]:
file_list <-  dir("/stor/public/chenyx/HHCAd/cell_type_organized_fetal/")[3:9]

In [8]:
file_list

[1] "Endothelial cell.rds"   "Fibroblast.rds"         "Lymphocyte.rds"        
[4] "Myelocyte.rds"          "Neuron.rds"             "Pericyte.rds"          
[7] "Smooth muscle cell.rds"

In [9]:
for(f in file_list){
    obj <-readRDS(paste0("/stor/public/chenyx/HHCAd/cell_type_organized_fetal/",f))
    if (dim(obj)[2]>2000){
    obj <- subset(obj, subset = cell_id %in% obj$cell_id[sample(1:dim(obj)[2],2000)])
    }
    obj_merge <- merge(obj_merge,obj)
}

In [10]:
table(obj_merge$cell_type)


         Adipocyte Cardiomyocyte cell   Endothelial cell         Fibroblast 
              1891               2000               2000               2000 
        Lymphocyte          Myelocyte             Neuron           Pericyte 
              1406               1558                958                355 
Smooth muscle cell 
              2000 

In [11]:
saveRDS(obj_merge,"/stor/public/chenyx/HHCAd/cell_type_organized_fetal/ForDEG.rds")

# prepare data

In [12]:
obj_merge <- readRDS("/stor/public/chenyx/HHCAd/cell_type_organized_fetal/ForDEG.rds")

In [13]:
df <- as.data.frame(as.matrix(obj_merge@assays$RNA@data))

In [14]:
df <- df[!rowSums(df)==0,]

In [15]:
label <- obj_merge$cell_type

In [16]:
df <- df[,!label=="NA"]
label <- label[!label=="NA"]

In [17]:
dim(df)

[1] 22912 14168

# build Obj

In [18]:
obj <- CreateSeuratObject(counts = df,min.cells = 0,min.features = 0)

In [19]:
obj$cell_type <- label

In [20]:
obj <- NormalizeData(obj, normalization.method = "LogNormalize", scale.factor = 10000)
obj <- ScaleData(obj)

Centering and scaling data matrix



In [21]:
Idents(obj) <- label

In [22]:
markers <- FindAllMarkers(obj,only.pos = TRUE,min.pct = 0.25, logfc.threshold = 0.25)

Calculating cluster Adipocyte

Calculating cluster Cardiomyocyte cell

Calculating cluster Endothelial cell

Calculating cluster Fibroblast

Calculating cluster Lymphocyte

Calculating cluster Myelocyte

Calculating cluster Neuron

Calculating cluster Pericyte

Calculating cluster Smooth muscle cell



In [23]:
ct_list <- unique(markers$cluster)

In [24]:
for(ct in ct_list){
    write.csv(markers[markers$cluster==ct,],paste0("/stor/public/chenyx/HHCAd/DEGene_cell_type_fetal/",ct,".csv"))
}